In [2]:
import pandas as pd
import numpy as np
import random
import copy
from collections import defaultdict

# Read data from CSV files
def load_data():
    courses_df = pd.read_csv('courses.csv')
    students_df = pd.read_csv('studentCourse.csv')
    students_names_df = pd.read_csv('studentNames.csv')
    teachers_df = pd.read_csv('teachers.csv')
    
    # Extract useful information
    course_codes = courses_df['Course Code'].unique()
    course_names = dict(zip(courses_df['Course Code'], courses_df['Course Name']))
    teachers = teachers_df['Names'].tolist()
    
    # Create student enrollment dictionary
    student_enrollments = defaultdict(list)
    for _, row in students_df.iterrows():
        student_enrollments[row['Student Name']].append(row['Course Code'])
    
    # Create course enrollments dictionary
    course_enrollments = defaultdict(list)
    for _, row in students_df.iterrows():
        course_enrollments[row['Course Code']].append(row['Student Name'])
    
    return {
        'course_codes': course_codes,
        'course_names': course_names,
        'teachers': teachers,
        'student_enrollments': student_enrollments,
        'course_enrollments': course_enrollments
    }

# Define time slots and days
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
time_slots = [f"{hour}:00" for hour in range(9, 17)]  # 9 AM to 4 PM (exams can run until 5 PM)
classrooms = [f"C{i}" for i in range(301, 311)]  # Classrooms C301 to C310

# Define the genetic algorithm components
class ExamScheduler:
    def __init__(self, data, population_size=50, generations=100, crossover_rate=0.8, mutation_rate=0.2):
        self.data = data
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.best_solution = None
        self.best_fitness = -1
        self.population = []
        
    def initialize_population(self):
        """Initialize random population of chromosomes"""
        self.population = []
        for _ in range(self.population_size):
            chromosome = self.create_random_chromosome()
            self.population.append(chromosome)
    
    def create_random_chromosome(self):
        """Create a random chromosome (schedule)"""
        chromosome = []
        used_slots = {}  # (day, time, classroom) -> course_code
        teacher_slots = {}  # (day, time) -> teacher
        
        for course in self.data['course_codes']:
            # Try to find a valid slot
            max_attempts = 50  # Limit attempts to prevent infinite loop
            assigned = False
            attempts = 0
            
            while not assigned and attempts < max_attempts:
                day = random.choice(days)
                time_slot = random.choice(time_slots)
                classroom = random.choice(classrooms)
                teacher = random.choice(self.data['teachers'])
                
                # Check if this slot is free
                slot_key = (day, time_slot, classroom)
                teacher_key = (day, time_slot)
                
                if slot_key not in used_slots and teacher_key not in teacher_slots:
                    # Add exam to schedule
                    exam = {
                        'course': course,
                        'day': day,
                        'time': time_slot,
                        'classroom': classroom,
                        'teacher': teacher
                    }
                    chromosome.append(exam)
                    used_slots[slot_key] = course
                    teacher_slots[teacher_key] = teacher
                    assigned = True
                
                attempts += 1
            
            # If couldn't find a valid slot after max attempts, force an assignment
            if not assigned:
                day = random.choice(days)
                time_slot = random.choice(time_slots)
                classroom = random.choice(classrooms)
                teacher = random.choice(self.data['teachers'])
                
                exam = {
                    'course': course,
                    'day': day,
                    'time': time_slot,
                    'classroom': classroom,
                    'teacher': teacher
                }
                chromosome.append(exam)
        
        return chromosome
    
    def calculate_fitness(self, chromosome):
        """Calculate fitness score for a chromosome"""
        # Initialize variables to track constraint violations
        hard_constraints_violations = 0
        soft_constraints_satisfied = 0
        
        # Track allocated slots
        used_slots = {}  # (day, time, classroom) -> course
        teacher_time_slots = {}  # (day, time) -> teacher
        student_time_slots = {}  # (student, day, time) -> course
        teacher_schedule = defaultdict(list)  # teacher -> [(day, time)]
        
        # Process each exam in the chromosome
        for exam in chromosome:
            course = exam['course']
            day = exam['day']
            time = exam['time']
            classroom = exam['classroom']
            teacher = exam['teacher']
            
            # Check slot allocation (Hard constraint 1)
            slot_key = (day, time, classroom)
            if slot_key in used_slots and used_slots[slot_key] != course:
                hard_constraints_violations += 1
            used_slots[slot_key] = course
            
            # Check teacher allocation (Hard constraint 5)
            teacher_key = (day, time)
            if teacher_key in teacher_time_slots and teacher_time_slots[teacher_key] != teacher:
                hard_constraints_violations += 1
            teacher_time_slots[teacher_key] = teacher
            
            # Track teacher schedule for consecutive duties check
            teacher_schedule[teacher].append((day, time))
            
            # Check for student exam conflicts (Hard constraint 2)
            hour = int(time.split(':')[0])
            if not (9 <= hour < 17):  # Check if exam is between 9 AM and 5 PM (Hard constraint 4)
                hard_constraints_violations += 1
                
            if day not in days:  # Check if exam is on weekday (Hard constraint 3)
                hard_constraints_violations += 1
            
            # Check student conflicts
            for student in self.data['course_enrollments'][course]:
                student_key = (student, day, time)
                if student_key in student_time_slots and student_time_slots[student_key] != course:
                    hard_constraints_violations += 1
                student_time_slots[student_key] = course
        
        # Check for consecutive teacher duties (Hard constraint 6)
        for teacher, schedule in teacher_schedule.items():
            # Sort schedule by day and time
            schedule.sort(key=lambda x: (days.index(x[0]), time_slots.index(x[1])))
            for i in range(len(schedule) - 1):
                day1, time1 = schedule[i]
                day2, time2 = schedule[i + 1]
                
                # Check if consecutive slots
                if day1 == day2:
                    idx1 = time_slots.index(time1)
                    idx2 = time_slots.index(time2)
                    if idx2 - idx1 == 1:  # Consecutive slots
                        hard_constraints_violations += 1
        
        # Check soft constraints
        
        # Soft constraint 1: Common break on Friday from 1-2 PM
        friday_1pm_exams = sum(1 for exam in chromosome if exam['day'] == 'Friday' and exam['time'] == '13:00')
        if friday_1pm_exams == 0:
            soft_constraints_satisfied += 1
        
        # Soft constraint 2: Students should not have back-to-back exams
        students_with_back_to_back = set()
        for student, courses in self.data['student_enrollments'].items():
            if len(courses) <= 1:
                continue
                
            # Get all exams for this student
            student_exams = []
            for exam in chromosome:
                if exam['course'] in courses:
                    student_exams.append(exam)
            
            # Check for back-to-back exams
            for i in range(len(student_exams)):
                for j in range(i + 1, len(student_exams)):
                    exam1 = student_exams[i]
                    exam2 = student_exams[j]
                    
                    if exam1['day'] == exam2['day']:
                        idx1 = time_slots.index(exam1['time'])
                        idx2 = time_slots.index(exam2['time'])
                        if abs(idx2 - idx1) == 1:  # Back-to-back
                            students_with_back_to_back.add(student)
        
        # If less than half of students have back-to-back exams, consider constraint satisfied
        if len(students_with_back_to_back) < len(self.data['student_enrollments']) / 2:
            soft_constraints_satisfied += 1
        
        # Soft constraint 3: MG courses before CS courses
        mg_cs_students = []
        for student, courses in self.data['student_enrollments'].items():
            has_mg = any(course.startswith('MG') for course in courses)
            has_cs = any(course.startswith('CS') for course in courses)
            if has_mg and has_cs:
                mg_cs_students.append(student)
        
        # Count students with MG courses scheduled before CS courses
        proper_order_count = 0
        for student in mg_cs_students:
            courses = self.data['student_enrollments'][student]
            mg_courses = [c for c in courses if c.startswith('MG')]
            cs_courses = [c for c in courses if c.startswith('CS')]
            
            # Get exam timings for these courses
            mg_exams = []
            cs_exams = []
            
            for exam in chromosome:
                if exam['course'] in mg_courses:
                    mg_exams.append((days.index(exam['day']), time_slots.index(exam['time'])))
                elif exam['course'] in cs_courses:
                    cs_exams.append((days.index(exam['day']), time_slots.index(exam['time'])))
            
            # Check if any MG exam is before any CS exam
            for mg_time in mg_exams:
                for cs_time in cs_exams:
                    if mg_time < cs_time:  # Day index lower or same day but earlier time
                        proper_order_count += 1
                        break
        
        # If majority of students have MG courses before CS courses
        if proper_order_count > len(mg_cs_students) / 2:
            soft_constraints_satisfied += 1
        
        # Soft constraint 4: Two-hour break for faculty meetings
        # Find a two-hour slot where at least half of faculty is free
        faculty_free_slots = defaultdict(set)
        
        for teacher in self.data['teachers']:
            # Assume all slots are free initially
            for day in days:
                for i in range(len(time_slots) - 1):  # Looking for two consecutive free hours
                    faculty_free_slots[(day, i)].add(teacher)
        
        # Remove slots where teachers are assigned
        for exam in chromosome:
            teacher = exam['teacher']
            day = exam['day']
            time_idx = time_slots.index(exam['time'])
            
            # Remove this slot and next slot (for 2-hour period)
            if (day, time_idx) in faculty_free_slots:
                faculty_free_slots[(day, time_idx)].discard(teacher)
            if (day, time_idx - 1) in faculty_free_slots:
                faculty_free_slots[(day, time_idx - 1)].discard(teacher)
        
        # Check if there's a slot where at least half the faculty is free
        half_faculty = len(self.data['teachers']) / 2
        faculty_meeting_possible = False
        
        for slot, free_teachers in faculty_free_slots.items():
            if len(free_teachers) >= half_faculty:
                faculty_meeting_possible = True
                break
        
        if faculty_meeting_possible:
            soft_constraints_satisfied += 1
            
        # Calculate final fitness score
        if hard_constraints_violations > 0:
            return 1 / (1 + hard_constraints_violations)  # Low fitness if hard constraints violated
        else:
            return 1 + soft_constraints_satisfied  # Max fitness is 5 (1 + 4 soft constraints)
    
    def selection(self):
        """Roulette wheel selection"""
        # Calculate fitness for all chromosomes
        fitness_scores = [self.calculate_fitness(chromosome) for chromosome in self.population]
        total_fitness = sum(fitness_scores)
        
        if total_fitness == 0:
            # If all chromosomes have zero fitness, select randomly
            return random.choice(self.population)
        
        # Calculate selection probabilities
        selection_probs = [fitness / total_fitness for fitness in fitness_scores]
        
        # Select a chromosome based on roulette wheel
        selected_idx = np.random.choice(range(len(self.population)), p=selection_probs)
        return self.population[selected_idx]
    
    def crossover(self, parent1, parent2):
        """Perform crossover between two parents"""
        if random.random() > self.crossover_rate:
            return copy.deepcopy(parent1)
        
        # Using one-point crossover adapted for our representation
        crossover_point = random.randint(1, len(self.data['course_codes']) - 1)
        
        # Create offspring using crossover
        child = []
        
        # Copy first part from parent1
        for i in range(crossover_point):
            child.append(copy.deepcopy(parent1[i]))
        
        # Copy remaining part from parent2, avoiding duplicate courses
        used_courses = set(exam['course'] for exam in child)
        
        for exam in parent2:
            if exam['course'] not in used_courses:
                child.append(copy.deepcopy(exam))
        
        # If we're missing any courses, add them from parent1
        remaining_courses = set(self.data['course_codes']) - used_courses
        
        for exam in parent1:
            if exam['course'] in remaining_courses:
                child.append(copy.deepcopy(exam))
                remaining_courses.remove(exam['course'])
        
        return child
    
    def mutate(self, chromosome):
        """Apply mutation to a chromosome"""
        if random.random() > self.mutation_rate:
            return chromosome
        
        # Select a random exam to mutate
        mutation_idx = random.randint(0, len(chromosome) - 1)
        exam = chromosome[mutation_idx]
        
        # Decide what to mutate (day, time, classroom, or teacher)
        mutation_type = random.choice(['day', 'time', 'classroom', 'teacher'])
        
        if mutation_type == 'day':
            exam['day'] = random.choice(days)
        elif mutation_type == 'time':
            exam['time'] = random.choice(time_slots)
        elif mutation_type == 'classroom':
            exam['classroom'] = random.choice(classrooms)
        else:  # teacher
            exam['teacher'] = random.choice(self.data['teachers'])
        
        return chromosome
    
    def evolve(self):
        """Main evolution loop"""
        # Initialize population
        self.initialize_population()
        
        # Track progress
        fitness_history = []
        best_fitness_history = []
        
        # Evolution loop
        for generation in range(self.generations):
            # Evaluate current population
            fitness_scores = [self.calculate_fitness(chromosome) for chromosome in self.population]
            avg_fitness = sum(fitness_scores) / len(fitness_scores)
            fitness_history.append(avg_fitness)
            
            # Find best solution in this generation
            best_idx = np.argmax(fitness_scores)
            current_best_fitness = fitness_scores[best_idx]
            current_best = self.population[best_idx]
            
            # Update overall best if needed
            if current_best_fitness > self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_solution = copy.deepcopy(current_best)
            
            best_fitness_history.append(self.best_fitness)
            
            # Report progress
            if generation % 10 == 0:
                print(f"Generation {generation}: Avg Fitness = {avg_fitness:.4f}, Best Fitness = {self.best_fitness:.4f}")
                
                # Print top 3 individuals if requested
                if generation % 20 == 0:
                    sorted_indices = np.argsort(fitness_scores)[::-1]  # Descending order
                    print("\nTop 3 schedules in this generation:")
                    for i in range(min(3, len(sorted_indices))):
                        idx = sorted_indices[i]
                        print(f"  Individual {i+1} - Fitness: {fitness_scores[idx]:.4f}")
                    print()
            
            # Create new population
            new_population = []
            
            # Elitism - keep the best individual
            new_population.append(copy.deepcopy(current_best))
            
            # Fill the rest through selection, crossover, and mutation
            while len(new_population) < self.population_size:
                # Selection
                parent1 = self.selection()
                parent2 = self.selection()
                
                # Crossover
                offspring = self.crossover(parent1, parent2)
                
                # Mutation
                offspring = self.mutate(offspring)
                
                # Add to new population
                new_population.append(offspring)
            
            # Replace old population
            self.population = new_population
        
        # Final evaluation
        print(f"\nEvolution completed after {self.generations} generations")
        print(f"Best Fitness: {self.best_fitness:.4f}")
        
        # Check what constraints are satisfied
        self.analyze_solution(self.best_solution)
        
        return self.best_solution, fitness_history, best_fitness_history
    
    def analyze_solution(self, solution):
        """Analyze the solution to see which constraints are satisfied"""
        print("\nConstraints Analysis:")
        
        # Analysis variables
        used_slots = {}  # (day, time, classroom) -> course
        teacher_time_slots = {}  # (day, time) -> teacher
        student_time_slots = defaultdict(list)  # (student, day) -> [(time, course)]
        teacher_schedule = defaultdict(list)  # teacher -> [(day, time)]
        
        # Process each exam in the solution
        all_courses_scheduled = set()
        time_constraint_satisfied = True
        weekday_constraint_satisfied = True
        teacher_constraint_satisfied = True
        
        for exam in solution:
            course = exam['course']
            day = exam['day']
            time = exam['time']
            classroom = exam['classroom']
            teacher = exam['teacher']
            
            all_courses_scheduled.add(course)
            
            # Check time constraint
            hour = int(time.split(':')[0])
            if not (9 <= hour < 17):
                time_constraint_satisfied = False
                
            # Check weekday constraint
            if day not in days:
                weekday_constraint_satisfied = False
                
            # Check teacher constraint
            teacher_key = (day, time)
            if teacher_key in teacher_time_slots and teacher_time_slots[teacher_key] != teacher:
                teacher_constraint_satisfied = False
            teacher_time_slots[teacher_key] = teacher
            
            # Track teacher schedule
            teacher_schedule[teacher].append((day, time))
            
            # Track student schedules
            for student in self.data['course_enrollments'][course]:
                student_time_slots[(student, day)].append((time, course))
        
        # Check consecutive teacher duties
        consecutive_duties = False
        for teacher, schedule in teacher_schedule.items():
            # Sort schedule by day and time
            schedule.sort(key=lambda x: (days.index(x[0]), time_slots.index(x[1])))
            for i in range(len(schedule) - 1):
                day1, time1 = schedule[i]
                day2, time2 = schedule[i + 1]
                
                # Check if consecutive slots on the same day
                if day1 == day2:
                    idx1 = time_slots.index(time1)
                    idx2 = time_slots.index(time2)
                    if idx2 - idx1 == 1:  # Consecutive slots
                        consecutive_duties = True
                        break
        
        # Check student overlaps
        student_overlaps = False
        for (student, day), exams in student_time_slots.items():
            if len(exams) <= 1:
                continue
                
            # Sort exams by time
            exams.sort()
            
            # Check for same time
            for i in range(len(exams) - 1):
                time1 = exams[i][0]
                time2 = exams[i + 1][0]
                if time1 == time2:
                    student_overlaps = True
                    break
        
        # Check soft constraints
        
        # Soft constraint 1: Common break on Friday from 1-2 PM
        friday_1pm_exams = sum(1 for exam in solution if exam['day'] == 'Friday' and exam['time'] == '13:00')
        friday_break = friday_1pm_exams == 0
        
        # Soft constraint 2: Students should not have back-to-back exams
        back_to_back_count = 0
        total_students = len(self.data['student_enrollments'])
        
        for (student, day), exams in student_time_slots.items():
            if len(exams) <= 1:
                continue
                
            # Sort exams by time
            exams.sort()
            
            # Check for back-to-back
            for i in range(len(exams) - 1):
                time1 = exams[i][0]
                time2 = exams[i + 1][0]
                
                idx1 = time_slots.index(time1)
                idx2 = time_slots.index(time2)
                
                if idx2 - idx1 == 1:  # Back-to-back
                    back_to_back_count += 1
                    break
        
        few_back_to_back = back_to_back_count < total_students / 2
        
        # Soft constraint 3: MG courses before CS courses
        mg_cs_students = []
        proper_order_count = 0
        
        for student, courses in self.data['student_enrollments'].items():
            has_mg = any(course.startswith('MG') for course in courses)
            has_cs = any(course.startswith('CS') for course in courses)
            if has_mg and has_cs:
                mg_cs_students.append(student)
                
                # Get all exams for this student
                mg_exams = []
                cs_exams = []
                
                for day in days:
                    if (student, day) in student_time_slots:
                        for time, course in student_time_slots[(student, day)]:
                            if course.startswith('MG'):
                                mg_exams.append((days.index(day), time_slots.index(time)))
                            elif course.startswith('CS'):
                                cs_exams.append((days.index(day), time_slots.index(time)))
                
                # Check if any MG exam is before any CS exam
                proper_order = False
                for mg_day, mg_time in mg_exams:
                    for cs_day, cs_time in cs_exams:
                        if mg_day < cs_day or (mg_day == cs_day and mg_time < cs_time):
                            proper_order = True
                            break
                    if proper_order:
                        break
                        
                if proper_order:
                    proper_order_count += 1
        
        mg_cs_order = proper_order_count > len(mg_cs_students) / 2 if mg_cs_students else True
        
        # Soft constraint 4: Two-hour break for faculty meetings
        faculty_free_slots = defaultdict(set)
        
        for teacher in self.data['teachers']:
            # Assume all slots are free initially
            for day in days:
                for i in range(len(time_slots) - 1):  # Looking for two consecutive free hours
                    faculty_free_slots[(day, i)].add(teacher)
        
        # Remove slots where teachers are assigned
        for exam in solution:
            teacher = exam['teacher']
            day = exam['day']
            time_idx = time_slots.index(exam['time'])
            
            # Remove this slot and next slot (for 2-hour period)
            if (day, time_idx) in faculty_free_slots:
                faculty_free_slots[(day, time_idx)].discard(teacher)
            if (day, time_idx - 1) in faculty_free_slots:
                faculty_free_slots[(day, time_idx - 1)].discard(teacher)
        
        # Check if there's a slot where at least half the faculty is free
        half_faculty = len(self.data['teachers']) / 2
        faculty_meeting_possible = False
        
        for slot, free_teachers in faculty_free_slots.items():
            if len(free_teachers) >= half_faculty:
                faculty_meeting_possible = True
                break
        
        # Print analysis results
        print("\nHard Constraints:")
        print(f"1. All courses scheduled: {'YES' if len(all_courses_scheduled) == len(self.data['course_codes']) else 'NO'}")
        print(f"2. No student overlaps: {'YES' if not student_overlaps else 'NO'}")
        print(f"3. Weekday-only exams: {'YES' if weekday_constraint_satisfied else 'NO'}")
        print(f"4. Exams between 9 AM - 5 PM: {'YES' if time_constraint_satisfied else 'NO'}")
        print(f"5. One teacher per exam without conflicts: {'YES' if teacher_constraint_satisfied else 'NO'}")
        print(f"6. No consecutive invigilation duties: {'YES' if not consecutive_duties else 'NO'}")
        
        print("\nSoft Constraints:")
        print(f"1. Friday 1-2 PM break: {'YES' if friday_break else 'NO'}")
        print(f"2. Minimal back-to-back exams: {'YES' if few_back_to_back else 'NO'}")
        print(f"3. MG courses before CS courses: {'YES' if mg_cs_order else 'NO'}")
        print(f"4. Faculty meeting breaks: {'YES' if faculty_meeting_possible else 'NO'}")
        
        # Count fulfilled constraints
        hard_fulfilled = sum([
            len(all_courses_scheduled) == len(self.data['course_codes']),
            not student_overlaps,
            weekday_constraint_satisfied,
            time_constraint_satisfied,
            teacher_constraint_satisfied,
            not consecutive_duties
        ])
        
        soft_fulfilled = sum([
            friday_break,
            few_back_to_back,
            mg_cs_order,
            faculty_meeting_possible
        ])
        
        print(f"\nTotal: {hard_fulfilled}/6 hard constraints and {soft_fulfilled}/4 soft constraints fulfilled")
    
    def print_schedule(self, solution):
        """Print the exam schedule in a formatted way"""
        print("\n======================= EXAM SCHEDULE =======================")
        
        # Sort by day and time for better readability
        sorted_schedule = sorted(solution, key=lambda x: (days.index(x['day']), time_slots.index(x['time']), x['course']))
        
        # Group by day
        schedule_by_day = defaultdict(list)
        for exam in sorted_schedule:
            schedule_by_day[exam['day']].append(exam)
        
        # Print schedule
        for day in days:
            print(f"\n{day}:")
            print(f"{'Time':<8} {'Course':<10} {'Classroom':<10} {'Teacher':<25}")
            print("-" * 60)
            
            day_exams = schedule_by_day.get(day, [])
            
            for exam in day_exams:
                print(f"{exam['time']:<8} {exam['course']:<10} {exam['classroom']:<10} {exam['teacher']:<25}")
        
        print("\n=============================================================")

# Main execution function
def run_exam_scheduler():
    # Load data
    print("Loading data...")
    data = load_data()
    
    # Initialize and run the genetic algorithm
    print("\nInitializing genetic algorithm...")
    scheduler = ExamScheduler(
        data,
        population_size=100,
        generations=200,
        crossover_rate=0.8,
        mutation_rate=0.2
    )
    
    print("\nEvolving exam schedule...")
    best_solution, fitness_history, best_fitness_history = scheduler.evolve()
    
    # Print the final schedule
    scheduler.print_schedule(best_solution)
    
    # Print evolution statistics
    print("\nEvolution Statistics:")
    print(f"Final best fitness: {scheduler.best_fitness:.4f}")
    print(f"Hard constraints satisfied: {6 if scheduler.best_fitness > 1 else 'Not all'}")
    print(f"Soft constraints satisfied: {int(scheduler.best_fitness - 1) if scheduler.best_fitness > 1 else 0}/4")
    
    return best_solution, fitness_history, best_fitness_history

# Execute the scheduler
if __name__ == "__main__":
    best_solution, fitness_history, best_fitness_history = run_exam_scheduler()

Loading data...

Initializing genetic algorithm...

Evolving exam schedule...
Generation 0: Avg Fitness = 3.1883, Best Fitness = 5.0000

Top 3 schedules in this generation:
  Individual 1 - Fitness: 5.0000
  Individual 2 - Fitness: 5.0000
  Individual 3 - Fitness: 5.0000



KeyboardInterrupt: 